In [5]:
import openai
import groq
import os
import json
from groq import Groq
from llama_index.core import SimpleDirectoryReader


In [6]:
os.environ["GROQ_API_KEY"] = "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"

# Summarize Files


In [8]:
reader = SimpleDirectoryReader(input_dir=".")
documents = reader.load_data()
doc_dicts = [{"content": d.text, **d.metadata} for d in documents]

In [9]:
doc_dicts

[{'content': 'MIT License\n\nCopyright (c) 2024 Ajay Arasanipalai\n\nPermission is hereby granted, free of charge, to any person obtaining a copy\nof this software and associated documentation files (the "Software"), to deal\nin the Software without restriction, including without limitation the rights\nto use, copy, modify, merge, publish, distribute, sublicense, and/or sell\ncopies of the Software, and to permit persons to whom the Software is\nfurnished to do so, subject to the following conditions:\n\nThe above copyright notice and this permission notice shall be included in all\ncopies or substantial portions of the Software.\n\nTHE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR\nIMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,\nFITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE\nAUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER\nLIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWI

In [10]:
PROMPT = f"""
The following is a list of file contents, along with their metadata. For each file, provide a summary of the contents.

{doc_dicts}

Return a JSON list with the following schema:

```json
{{
  "files": [
    {{
      "filename": "name of the file",
      "summary": "summary of the content"
    }}
  ]
}}
```
""".strip()

In [11]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "Always return JSON. Do not include any other text or formatting characters.",
        },
        {
            "role": "user",
            "content": PROMPT,
        },
    ],
    model="llama3-70b-8192",
    response_format={"type": "json_object"},
)

summaries = json.loads(chat_completion.choices[0].message.content)["files"]

In [12]:
summaries

[{'filename': 'LICENSE', 'summary': 'MIT license information'},
 {'filename': 'loader.py',
  'summary': 'Function definition for getting document summaries'},
 {'filename': 'requirements.txt',
  'summary': 'List of dependencies for the project'},
 {'filename': 'scratch.ipynb',
  'summary': 'Code for interacting with LLaMA and Groq'},
 {'filename': 'scratch.ipynb',
  'summary': 'Code for interacting with LLaMA and Groq'},
 {'filename': 'scratch.ipynb',
  'summary': 'Code for interacting with LLaMA and Groq'}]

# Create File Tree


In [13]:
PROMPT = f"""
The following is a list of files and a summary of their contents. Read them carefully, then propose a directory structure that optimally organizes the files using known conventions and best practices.

{summaries}

You will solve this task by adding a `path` key to the JSON object below. The value of the `path` key should be the path to the file that you think is the most relevant to the summary.
""".strip()

In [14]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "Always return JSON. Do not include any other text or formatting characters.",
        },
        {
            "role": "user",
            "content": PROMPT,
        },
    ],
    model="llama3-70b-8192",
    # response_format={"type": "json_object"},
)

file_tree = json.loads(chat_completion.choices[0].message.content)

In [16]:
import pathlib

In [24]:
BASE_DIR = pathlib.Path("test_dir")
BASE_DIR.mkdir(exist_ok=True)

for file in file_tree:
    file["path"] = pathlib.Path(file["path"])
    # Create file in specified base directory
    (BASE_DIR / file["path"]).parent.mkdir(parents=True, exist_ok=True)
    with open(BASE_DIR / file["path"], "w") as f:
        f.write("")

In [26]:
json.loads('{\n        "file_path": "/Users/iyaja/Git/llama-fs/LICENSE",\n        "summary": "MIT License"\n    },\n    {\n        "file_path": "/Users/iyaja/Git/llama-fs/main.py",\n        "summary": "Python script"\n    },\n    {\n        "file_path": "/Users/iyaja/Git/llama-fs/requirements.txt",\n        "summary": "List of dependencies"\n    },\n    {\n        "file_path": "/Users/iyaja/Git/llama-fs/scratch.ipynb",\n        "summary": "Jupyter notebook"\n    }')

JSONDecodeError: Extra data: line 4 column 6 (char 98)